In [1]:
import tweepy
from collections import Counter
from collections import defaultdict
import random
import pandas as pd
import time

In [2]:
consumer_key = 'MivVnKovNAIOFxYgWAWnt2UW9'
consumer_secret = 'hIOLhbIaJ083XgC4kZIYDow0VBS3u5oJ28lK8NNtZcpxGWs3qp'
access_token = '11518572-sHaaBIRgwtITXLOmbXoo3PxeIrxJRgAb0kCF2MJrk'
access_secret = 'Qo0f6A1xermF94mZUKExd1x7xTw2vruEVtj7AOGJ2Y49a'
 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, retry_count=3, retry_delay=5, retry_errors=set([401, 404, 500, 503]))

In [3]:
def find_hashtags(hashtag):

    hashtags = Counter()

    try:
        for tweet in tweepy.Cursor(api.search, q='#'+hashtag, count=200).items():
            # for each hashtag in tweet
            for hashtag_in_tweet in [tag['text'] for tag in tweet.entities['hashtags']]:
                if not hashtag_in_tweet in hashtags:
                    hashtags[hashtag_in_tweet] = 1
                else:
                    hashtags[hashtag_in_tweet] += 1
    except tweepy.error.TweepError:
        pass
        time.sleep(5) # provide a time before resuming the code when an error arises
                                
    return hashtags

In [4]:
def clusters(hashtag, count=1):
    
    def find_hashtag_clusters(hashtag, count=1):

        if count <= 10:
            ht = find_hashtags(hashtag)

            new.update(ht)
            current.update(ht)
            return find_hashtag_clusters(random.choice(new.most_common(30))[0], count+1)
        else:
            ht = find_hashtags(hashtag)

            new.update(ht)

            if [i[0] for i in current.most_common(30)] == [i[0] for i in new.most_common(30)]:
                return new
            else:
                current.update(ht)
                return find_hashtag_clusters(random.choice(new.most_common(30))[0], count+1)

    current = Counter()
    new = Counter()
    
    find_hashtag_clusters(hashtag, count=1)
            
    return new

In [5]:
def get_profile(user_profile):

    profile = {}

    profile['user_bg_image'] = user_profile.profile_background_image_url
    profile['user_id'] = user_profile.id_str
    profile['user_profile_image'] = user_profile.profile_image_url
    profile['user_created_date'] = user_profile.created_at
    profile['user_followers_count'] = user_profile.followers_count
    profile['user_geo_enabled'] = user_profile.geo_enabled
    profile['user_tweet_count'] = user_profile.statuses_count
    profile['user_location'] = user_profile.location
    profile['user_verified'] = user_profile.verified
    profile['user_time_zone'] = user_profile.time_zone
    profile['user_name'] = user_profile.name
    profile['user_url'] = user_profile.url
    profile['user_screen_name'] = user_profile.screen_name
    profile['user_language'] = user_profile.lang
    profile['user_friends_count'] = user_profile.friends_count
    profile['user_description'] = user_profile.description
    profile['user_listed_count'] = user_profile.listed_count
    profile['userID'] = user_profile.id
    
    return profile

In [6]:
def get_tweet_author_profiles(tweet_id):
    
    # Get the user IDs for the original author and first 100 retweeters
    profile_ids = [api.get_status(tweet_id).user.id]   
    profile_ids.extend([x.user.id for x in api.retweets(tweet_id)])
    
    data = defaultdict()
    
    # Get their profiles and the profiles of their followers
    for userid in profile_ids:
        
        profile = get_profile(api.get_user(userid))
        
        data[profile['user_screen_name']] = profile
    # Return list of profiles
    return data

In [ ]:
def find_profiles_from_hashtag_clusters(hashtag):
    
    targets = defaultdict()
    
    hashtags = [i[0] for i in clusters(hashtag).most_common(30)]
    try:
        for tag in hashtags:
            time.sleep(5) # provide a time before resuming the code when an error arises
            print(len(targets))
            
                time.sleep(5) # provide a time before resuming the code when an error arises
                targets.update(get_tweet_author_profiles(tweet.id))
                print(len(targets))

    except tweepy.error.TweepError:
            passfor tweet in tweepy.Cursor(api.search, q='#'+tag, count=200).items():
            
    return targets

In [ ]:
data = find_profiles_from_hashtag_clusters('whitegenocide')

Rate limit reached. Sleeping for: 507


In [ ]:
len(df)

In [ ]:
df = pd.DataFrame.from_dict(data, orient='index')
df.to_csv('data.csv')